# (0) Download Butterfly 

In [14]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species

 99%|███████████████████████████████████████▌| 395M/399M [00:11<00:00, 44.1MB/s]
100%|████████████████████████████████████████| 399M/399M [00:11<00:00, 36.1MB/s]


In [1]:
import zipfile

with zipfile.ZipFile('butterfly-images40-species.zip', 'r') as zip_ref:
    zip_ref.extractall('data/butterfly')

# (1) Process Butterfly Data

In [1]:
import matplotlib.pyplot as plt

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

from tqdm import trange
import numpy as np
import os
import random

In [2]:
# Dataset Class
class ImageDataset(Dataset):
    def __init__(self, path_label, transform=None):
        self.path_label = path_label
        self.transform = transform

    def __len__(self):
        return len(self.path_label)

    def __getitem__(self, idx):
        path, label = self.path_label[idx]
        img = Image.open(path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img, label
    
# Create dataset from raw structure obtained from Kaggle
# (1): Download dataset fron Kaggle: https://www.kaggle.com/datasets/gpiosenka/butterfly-images40-species?select=train
# (2): Unzip the downloaded dataset to './data/butterfly'
# (3): Run the function
def create_dataset(path = './data/butterfly/'):
    transform = transforms.Compose([
                                    transforms.Resize((224,224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    train_path = path + 'train'
    test_path = path + 'test'
    
    class_names=sorted(os.listdir(train_path))
    N=list(range(len(class_names)))
    normal_mapping=dict(zip(class_names,N)) 
    reverse_mapping=dict(zip(N,class_names))

    paths0=[]
    for dirname, _, filenames in os.walk(train_path):
        for filename in filenames:
            if filename[-4:]=='.jpg':
                path=os.path.join(dirname, filename)
                label=dirname.split('\\')[-1]
                if label == '.ipynb_checkpoints':
                    continue
                paths0+=[(path,normal_mapping[label])]
            
    tpaths0=[]
    for dirname, _, filenames in os.walk(test_path):
        for filename in filenames:
            if filename[-4:]=='.jpg':
                path=os.path.join(dirname, filename)
                label=dirname.split('\\')[-1]
                if label == '.ipynb_checkpoints':
                    continue
                tpaths0+=[(path,normal_mapping[label])]

    random.shuffle(paths0)            
    random.shuffle(tpaths0)  

    trainset = ImageDataset(paths0, transform)
    testset = ImageDataset(tpaths0, transform)
    
    return trainset, testset, normal_mapping, reverse_mapping


trainset, testset, normal_mapping, reverse_mapping = create_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

# (2) Classifier

In [3]:
# Convolutional block with layer norm
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dropout, padding=0, stride=1, bias=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=bias)
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        self.dropout = nn.Dropout2d(p=dropout)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.dropout(x)
        return x

# CNN Classifier
class Classifier(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = ConvBlock(in_channels=in_channels, out_channels=64, kernel_size=3, dropout=0.2, padding=1, stride=1, bias=True)
        self.conv2 = ConvBlock(in_channels=64, out_channels=128, kernel_size=3, dropout=0.2, padding=1, stride=1, bias=True)
        self.conv3 = ConvBlock(in_channels=128, out_channels=256, kernel_size=3, dropout=0.2, padding=1, stride=1, bias=True)
        self.fc1 = torch.nn.LazyLinear(out_features=16)
        self.fc2 = torch.nn.LazyLinear(out_features=64)        
        self.fc3 = torch.nn.LazyLinear(out_features=num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.relu(self.conv3(x))
        
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)
        return logits
    

In [6]:
def train(model, train_loader, test_loader, optimizer, criterion, epoch, device):
    model = model.to(device)
    criterion = criterion.to(device)
    
    train_loss = 0
    test_loss = 0
    
    model.train()
    for batch_idx, (inp, label) in enumerate(train_loader):
        optimizer.zero_grad()
        inp, label = inp.to(device), label.to(device)
        output = model(inp)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (inp, label) in enumerate(test_loader):
            inp, label = inp.to(device), label.to(device)
            output = model(inp)
            loss = criterion(output, label)
            test_loss += loss.item()
    return train_loss / len(train_loader), test_loss / len(test_loader)
    

# Params
batch_size=100
lr = 0.0001
device='cuda'

# Data loader
train_loader = DataLoader(dataset=trainset, batch_size=batch_size)
test_loader = DataLoader(dataset=testset, batch_size=batch_size)

# Model
model = Classifier(in_channels=3, num_classes=len(normal_mapping))
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

# load states
# model.load_state_dict(torch.load('butterfly_classifier.pth'))

In [ ]:
train_hist = list()
test_hist = list()
for epoch in trange(1, 50 + 1):
    train_loss, test_loss = train(model, train_loader, test_loader, optimizer, criterion, epoch, device=device)
    train_hist.append(train_loss)
    test_hist.append(test_loss)
    print('Epoch {}: Train: {}, Test: {}'.format(epoch, train_loss, test_loss))
    
torch.save(model.state_dict(), 'butterfly_classifier.pth')

  2%|█▋                                                                                 | 1/50 [00:32<26:50, 32.86s/it]

Epoch 1: Train: 2.277302070269509, Test: 2.411353874206543


  4%|███▎                                                                               | 2/50 [01:05<26:13, 32.78s/it]

Epoch 2: Train: 2.153516245266748, Test: 2.355256509780884


  6%|████▉                                                                              | 3/50 [01:38<25:36, 32.70s/it]

...

Epoch 47: Train: 0.10527328899987633, Test: 5.325395488739014


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [26:30<01:05, 32.66s/it]

Epoch 48: Train: 0.09440573510373869, Test: 5.468943500518799


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [27:03<00:32, 32.75s/it]

Epoch 49: Train: 0.09699635444179414, Test: 5.425824737548828
